![](CrowdsourcingJ.png)

##                              CROWDSOUCING JUSTICE 
##                             -by Rafael "Rafa" V. Baca, Esq.


PREFACE: 
Historically, Immigration Law has been our nation's bellwether indicator for ensuring our inalienable, constitutionally-ensured, human rights and access to justice in our democratic society.  Current events have challenged these democratic freedoms that should never be taken for granted as immigrants are being turned away from our lands. Additionally, a growing number of poor and vulnerable populations in the U.S. are economically restricted from legal representation (1 lawyer for every 5000 legally harmed), and, thus, direct access to justice.

MISSION STATEMENT: 
This project is an active continuation of my recent SXSW talk: "Crowdsourcing Justice" where the key idea is to apply an "overarching" software architectures and machine learning models that will provide continuous pro bono legal services by a revolving workforce of Attorneys and interested volunteers for vulnerable populations that typically do not presently share equal access to justice. Through my startup, Torchlight Legal, Pro Bono attorneys are given machine augmented litigation support, such as automated forms and legal research, to ensure existing cases are a success while increasing population of legal represented individuals.  Through a simple phone app, Torchlight Legal acts as the user interface between volunteer lawyers and supporting professionals in California, Texas, and New York and those vulnerable individuals who may be hesitant to go directly to law enforcement authorities or employers for help.

INTITIAL PROJECTS [OPTIONAL - A MACHINE LEARNING USE CASE]: 

CASETEXT of San Francisco has provided use of the US Immigration (Administrative) Court’s body of legal cases so that Torchlight Legal will derive a dataset and create a database infrastructure for machine learning models to predict a court outcome based on prior rulings by a particular judge.  Accordingly, this project will demonstrate a "proof-of-concept" electronic database architecture for volunteer legal services intially within the states of California (One Justice) and Texas (RioGrande Legal Aid) to help pro bono clients in many categories of law beyond immigration law.   


![](OneJustice.png)

## PROJECT ARCHITECTURE 
## -- DIRECTED ACYCLIC GRAPHS, DAGs --






## 1.)  FINAL PROJECT INFRASTRUCTURE "PROOF-OF-CONCEPT":

[Optional - Technical Explination of Big Data Architecture]



![](ArchiTLL.jpg)

## 2.)  ORIGINAL PROJECT INFRASTRUCTURE PROTOTYPES:

[Optional - Technical Explination of Big Data Architecture]

Technical Specifications of Infrastructue - 

PROTOTYPE 2:  
As the precursor to the above, "proof-of-concept" model, the below shows multiple datasources including streaming datasources.  After much research and individual solicitation, streaming datasources specifically directed toward the legal field were not in existance.  Notably, to account of streaming, Kensis, was added in to below architecture.  Staged data refers to a Relational Database Services provided by Postgres on AWS.  Accordingly, the tables of legal data would be queried via Postgres.

![](DAG01ver.png)

Technical Specifications of Infrastructue - 

PROTOTYPE 1:  
The very first prototype shown below is an early architecture for a legal volunteer database.  In operation, a volunteer fills out a form on the Torchlight Legal website or app while on the internet layer via a local client, shown as a laptop.  The form shown below is based on a Google Docs template or equivilant technology.  On a continuing basis, data is retrieved from the fields of the volunteer forms by an EC2 instance on AWS.  The data is collected in S3 and then transformed to a resiliantly distributed dataset, RDD, via Postgress on the EC2 instance.  As the volunteer data is being collected, an email notification of receiving new volunteers is directly sent from google email hosting to the membership manager's inbox at Torchlight Legal. 

![](DAG052of2.png)

## --------------- CODE 1 - Data to S3 ---------------
The Following Code Transfers a local file to AWS S3.  The local file contains about 4000 recent law cases each with a legal ruling.  Each law case is formatted in XML file.

In [ ]:
import boto
import os
import pandas as pd
import ssl
import sys
from boto.s3.connection import S3Connection

def main():
    '''
    This pulls xml casefiles
    1. Import credentals
    2. Retrieve Casefiles
    3. Loop Casefiles to Push to s3
    '''
    pass

# AWS Credentials
S3Conn = S3Connection('MyAWSaccessKey','MyAWSsecretKey')
greatbucket =S3Conn.get_bucket('bucketname')

###  Accessing Desktop Folder
path = '/Users/mycomputerName/Desktop/myFolder/'

# UPLOAD BY LOOPING THRU BY INDIVIDUAL XML FILE
for i in (os.listdir(path)):
     file = greatbucket.new_key(i)
     file.set_contents_from_filename(path+'/'+i)

if __name__ == '__main__':
     main()


## --------------- CODE 2  - S3 to RDS ----------------
With an AWS EC2 instance executed by pyspark (Apache Spark's dialect of python), the following code transforms the case law data initially stored on S3 to become a RDD, resiliently distributated dataset.  Prior to this RDD transformation, tables (aspiring to be in the 3rd Normal form) where created on a Posgres (pgAdmin4) Relational Database Server (RDS). The RDS tables receive a parsed version of the S3 case law data that is executed by the following program:





The TABLES are listed as follows:
*******************************************
TABLE 1   
CASE
Tag/col name 	dtype/postgres          KEY / Descrption
docket      		 varhar    50		PRIMARY KEY/CASE Docket 
reporter_caption	varchar 200		reporter citation (with yr and ct)
date			   Date			Decision  Full Date  
opinion_byline	text  			Presiding Judge
bold			text			Case Holding/Ruling


TABLE 2
OPINION
Tag/col name 	dtype/postgres          KEY / Descrption
docket      		 varhar    50		PRIMARY KEY/CASE Docket     len 50 
opinion_text		text			full text of case
(page_number	smallINT		pp of opinion) - Optional
footnote_number	smallINT		footnotes no. in case
footnote_body	varchar		full text of footnotes

TABLE 3
CITED CASES
Tag/col name 	dtype/postgres          KEY / Descrption
docket      		 varhar    50		PRIMARY KEY/CASE Docket     len 50 
cross_reference	varchar   250		case and codified legal citations

TABLE 4
PLAYERS
Tag/col name 	dtype/postgres          KEY / Descrption
docket      		 varhar     50		PRIMARY KEY/CASE Docket     len 50 
panel			text			Judges on Bench
attorneys		text			Plaintiff and Govt	

TABLE 5 (OPTIONAL)
DISSENT
Tag/col name 	dtype/postgres          KEY / Descrption
docket      		 varhar    50		PRIMARY KEY/CASE Docket     len 50 
dissent_byline	text			judges writing byline
dissent_text		text			dissent text of opinion
*******************************************

In [6]:
import sys
from bs4 import BeautifulSoup
import pandas as pd
import boto
from boto.s3.connection import S3Connection
import os
import ssl
from pyspark import SparkContext
from pyspark.sql import SparkSession
import psycopg2
import string

###########
# AWS Credentials
#S3Conn = S3Connection('MyAWSaccessKey','MyAWSsecretKey')

# ------------ STEP ----------------#
S3Conn = S3Connection('MyAWSaccessKey', 'MyAWSsecretKey',
                      host='s3.amazonaws.com')
greatbucket =S3Conn.get_bucket('bucketname')
#greatbucket =S3Conn.get_bucket('testbucket')


#spark context initialization & spark SQL initialize
sc = SparkContext("local[*]")
spark = SparkSession(sc)

# assigning soup as RDD
path = "s3://bucket/"
#path = "s3://testbucket/"

#  --------STEP -------------#
# create database connection
dbConn = psycopg2.connect(**{
         'dbname':'dbname',
         'host':'host.usa-9.rds.amazonaws.com',
         'user':'user',
         'password':'pwd'
         })
cur = dbConn.cursor()
#cur.execute('INSERT INTO "CASE" VALUES (%s, %s, now(), %s, %s)', ("1", "2", "3", "4") )
#dbConn.commit()

def main():
    '''
    This pulls xml casefiles on S3 bucketname bucket
    1. Import credentals
    2. Loop Casefiles to retrieve contents
    3. while looping parsing and storing conents in dictionaries for corrsponding table
    4.  generating a RDD on S3 based on dictionary values
    '''
    print( "PATH: ", path )
    partition( path )
    #partition( "foldername" )
    #pass

# function created for list of files: os.listdir(path)
# function to read xml files
def partition(path):
    counterDocket = 0
    counterDuplicate = 0
    counterSkip = 0
    dockets = {}

    #for i in (os.listdir(path)):
    for key in greatbucket.list():
            # if true or case with many names
        if 1==1 or key.name == 'singlecaseName.xml':
            #print("MYBUCKET: ",  key.name.encode('utf-8') )
            print("MYFILE: ",  key.name )
            #print("MYCONTENT: ",  key.get_contents_as_string() )

            soup = BeautifulSoup( key.get_contents_as_string() , 'xml')

            dataCASE = {
                'docket' : '',
                'reporter_caption' : '',
                'date' : '',
                'opinion_text' : '',
                'find_order' : ''
            }

            cols = soup.find_all('docket')

            dataCASE['docket'] = None
            if len(cols) > 0:

                # checking for spaces (on either side) on docket
                # also works to specifically erase unwanted char- lft string
                col0Trimmed = cols[0].get_text().lstrip()
                if col0Trimmed.split(" ") > 1:
                    dataCASE['docket'] = col0Trimmed.split(" ")[0]
                else:
                    dataCASE['docket'] = col0Trimmed

                # checking for dots on docket
                if dataCASE['docket'].split(".") > 1:
                    dataCASE['docket'] = dataCASE['docket'].split(".")[0]

                #print("MYDOCKET w true: ", len(cols), dataCASE['docket'], dataCASE['docket'] in dockets)

            #print("SIZE: ", len(dockets) )
            #if dataCASE['docket'] in dockets:
                #print("IS DUPKICATE: ", dockets[dataCASE['docket']])
# RAW DATA - VERY PROBLEMATIC BEGINNING WITH "DOCKET"
            # if no docket, SKIP
            if len(cols) > 0 and dataCASE['docket'] is not None and not dataCASE['docket'] in dockets:

                #print("MYDOCKET IN: ", len(cols), dataCASE['docket'] )
                # to keep track of dockets
                counterDocket = counterDocket + 1
                dockets[dataCASE['docket']] = dataCASE['docket']
                #print("LENGTH XXX: ", len(dockets))

                cols = soup.find_all('reporter_caption')
                #print("MYREPORTER: ", len(cols), cols[0].get_text() )
                dataCASE['reporter_caption'] = cols[0].get_text()

                cols = soup.find_all('date')
                #print("MYDATE: ", len(cols), cols[0].get_text() )
                dataCASE['date'] = cols[0].get_text()

                cols = soup.find_all('opinion_text')
                if len(cols) > 0:
                    #print("MYOPINION: ", len(cols), cols[0].get_text() )
                    dataCASE['opinion_text'] = cols[0].get_text()
                else:
                    dataCASE['opinion_text'] = ''
                    #print("MYOPINION: ", len(cols) )

                # try to get order
                bold = find_order( soup )
                dataCASE['find_order'] = bold
                #print("MYORDER: ", len(cols), bold )

                # filter out binary data from find order / bold
                #stripping non printable characters from a string
                if dataCASE['find_order'] is not None:
                    dataCASE['find_order'] = filter(lambda x: x in string.printable, dataCASE['find_order'])

                # LOAD INTO DATABASE
                cur.execute('INSERT INTO "CASE" ' +
                    '(docket, reporter_caption, date, opinion_byline, bold) ' +
                    'VALUES (%s, %s, %s, %s, %s)', (
                        dataCASE['docket'],
                        dataCASE['reporter_caption'],
                        dataCASE['date'],
                        dataCASE['opinion_text'],
                        dataCASE['find_order']
                    ))
            else:
                if len(cols) <= 0:
                    counterSkip = counterSkip + 1
                else:
                    counterDuplicate = counterDuplicate + 1

            print("Dockets Loded: ", counterDocket)
            print("Dockets Duplicate : ", counterDuplicate)
            print("Dockets Skip : ", counterSkip)

    dbConn.commit()
    print("Dockets Loded: ", counterDocket)
    print("Dockets Duplicate : ", counterDuplicate)
    print("Dockets Skip : ", counterSkip)

#removing control characters/codes fm a string
# aka nonprintable to
# in this case it was <tag>[fn6]<tag>
#https://github.com/nlplab/brat/blob/master/server/src/realmessage.py

def remove_control_chars(s):
    return control_char_re.sub('', s)

# this function gets the Order for a Case
def find_order(s):
    result = None
    for i in s.find_all('bold'):
        #print("BOLD len: ", len(i))
        #print("BOLD: ", str(i))
        if str(i) == '<bold>ORDER:</bold>':
            result = i.next_sibling
            #print("GOT IN: ", len(i))
            #print("RESULT BOLD: ", result)
    #print("RESULT: ", result)
    return result


### TODO: THIS IS NOT BEING CALLED ###
def opinion():
    ##  Load opinion
    dataOPINION = {
        'opinion_text' : [],
        'page_number' : [],
        'footnote_number' : [],
        'footnote_body' : [],
    }

    for soup in soups:
        dataOPINION['docket'].append( cols[0].get_text() )
        cols = soup.find_all('opinion_text')
        dataOPINION['opinion_text'].append( cols[1].get_text() )
        cols = soup.find_all('page_number')
        dataOPINION['page_number'].append( cols[2].get_text() )
        cols = soup.find_all('footnote_number')
        dataOPINION['footnote_number'].append( cols[3].get_text() )
        cols = soup.find_all('footnote_body')
        dataOPINION['footnote_body'].append( cols[4].get_text() )
    ##
    dataCCASES = {
        'cross_reference' : []
    }

    for soup in soups:
        dataCCASES['docket'].append( cols[0].get_text() )
        cols = soup.find_all('cross_reference')
        dataCCASES['cross_reference'].append( cols[1].get_text() )
    ##
    dataPLAYERS = {
        'panel' : [],
        'attorneys' : []
    }

    for soup in soups:
        dataPLAYERS['docket'].append( cols[0].get_text() )
        cols = soup.find_all('panel')
        dataPLAYERS['panel'].append( cols[1].get_text() )
        cols = soup.find_all('attorneys')
        dataPLAYERS['attorneys'].append( cols[2].get_text() )
    ##
    dataDISSENT = {
        'dissent_byline' : [],
        'dissent_text' : []
    }

    for soup in soups:
        dataDISSENT['docket'].append( cols[0].get_text() )
        cols = soup.find_all('dissent_byline')
        dataDISSENT['dissent_byline'].append( cols[1].get_text() )
        cols = soup.find_all('dissent_text')
        dataDISSENT['dissent_text'].append( cols[2].get_text() )

    #load data into database
    #rdd.foreachPartition(soup)

#caseData = pd.DataFrame( dataCASE )
#caseData.to_csv("caseData.csv")
############

ModuleNotFoundError: No module named 'pyspark'

## TABLE "CASES" BUILD OUTPUT:

('Dockets Loded: ', 3298)

('Dockets Duplicate : ', 487)

('Dockets Skip : ', 181)

('MYFILE: ', u'test/')

('Dockets Loded: ', 3298)

('Dockets Duplicate : ', 487)

('Dockets Skip : ', 182)

('MYFILE: ', u'test/singlelawcase.xml')

('Dockets Loded: ', 3298)

('Dockets Duplicate : ', 488)

('Dockets Skip : ', 182)

('MYFILE: ', u'test/singlelawcase.xml')

('Dockets Loded: ', 3298)

('Dockets Duplicate : ', 489)

('Dockets Skip : ', 182)

('Dockets Loded: ', 3298)

('Dockets Duplicate : ', 489)

('Dockets Skip : ', 182)

## NOTES:  

## PYSPARK NOTES:  

Personally, I found Zeppelin to not execute as desired during the debugging process.  Pyspark was subsequently exectuted directly from the terminal.  

Here are some 
## helpful commands:

###########################
# to connect to zeppelin
ecListNumbersofDesiredEC2InstanceHere.compute1.amazonaws.com:8890

Plus Amazon Login

# copy data to ec2 instance
scp –i NameofYourPemKey.pem 3DraftLegalSoup.py
ec2-user@ecListNumberofDeseiredEC2InstanceDashes.compute-1.amazonaws.com :.

# copy data to ec2 instance
scp –r –i NameofYourPemKey.pem desiredBucketName ec2-user@ecListNuberofDesiredEC2InstanceDashes.compute-1.amazonaws.com :.    #Be sure not to forget "period" at end

# to connect to the instance
ssh –i NameofYourPEMKey.pem ec2user@ecListNumberNoDashofDesiredEC2Instance.compute1.amazonaws.com
terminal looks like (you shoud be direct find the instance connection
directly from the AWS ec2 Console:
ComputerNames-MacBook-Pro-2:Desktop x$ ssh -i NAME.pem ec2-user@ec2-ListNumberDesiredInstanceDashes.compute-1.amazonaws.com

# create aws config file with secret keys
mkdir ~/.aws
vi ~/.aws/config

# add the following to the config file:
[default]
aws_secret_access_key = AWSSECRET ACCESS KEY
aws_access_key_id = AWSACCESSKEY
output = json
region = SomeUSAplace-7

# pick first xml file: lawcasename.xml
ls –l | bucketName | head

# Install pandas on EC2 instance
sudo pip install –upgrade pip

# logout and log back in
sudo /usr/local/bin/pip install pandas

# install pyspark and py4j
sudo /usr/local/bin/pip install py4j

################ Running On pyspark ############
# USE sparksubmit
spark-submit 3DraftLegalSoup.py

[ec2-user@ip-9dashedNumbers ~]$ run python program

-bash: run: 

[ec2-user@ip-9dashedNumbers ~]$ spark-submit 3DraftLegalSoup.py

# to know if something failed use this:
echo $?


# to connect to postgresql from the terminal, use password: myDesiredPostgresServerPassword
psql –h bucketName.awsaddress.usa-someRegion-4.rds.amazonaws.com –U bucketName

	
# PLEASE find a terminal editor you like
## --- To OPEN/&/CREATE FILES TYPE:
Terminal editors abound, can be easily found with webrowser search:  Some editors tried - Nano, Emacs, & Vi.
$   nano 3DraftLegalSoup.py
$   emacs 3DraftLegalSoup.py
$   vi 3DraftLegalSoup.py


# how to edit the program

NANO Editor – 
(COMMAND menu is given at the very bottom of the Editor in black
BE sure to REMEMBER that ^X  the ^caret^ means “CONTROL” in MAC

save the changes you've made, press Ctrl + O . 
EXIT nonModified -  nano, type Ctrl + X . 
EXIT from a modified file: it will ask you if you want to save it:  Y to Save, N not save - will then ask you for a filename.
NANO-


https://wiki.gentoo.org/wiki/Nano/Basics_Guide 
https://www.lifewire.com/beginners-guide-to-nano-editor-3859002 

Posgres:
USE TRUNCATE (Right CLICK on Desired Table – Truncate option) !!USE TRUNCATE AFTER EACH TIME YOU EDIT PROGRAM as NEW DATA NEEDS TO BE LOADED PURSUANT TO NEWEST CODE COMMANDS!!  TRUNCATE –quickly removes all rows from a set of tables. It has the same effect as an unqualified DELETE on each table, but because it does not actually scan the tables it is faster. Furthermore, it reclaims disk space immediately, rather than requiring a subsequent VACUUM operation. This is most useful on large tables.

Right Click on Desired Table  – click PROPERTIES at Bottom  to review columns  -- to increase the size of databin

RESTORE – Right Click on Desired Table – restores a backup database back on to S3.  http://stackoverflow.com/questions/2732474/restore-a-postgres-backup-file-using-the-command-line 

Posgres SQL:
http://technobytz.com/most-useful-postgresql-commands.html 


## ----- CODE 3 - RDS to QUERY & PUBLICATION -----
With POSTGRES linked to the presently active AWS EC2 instance: 

1) A SQL qury was first applied to determine the number of unique Judicial rulings are contained in the present database. 

2) A python program was developed to operate on a csv file derived from the present database.  This python program counts the number of words provided in the judicial rulings and posts the result on an AWS WEBSITE HERE: https://s3.amazonaws.com/output6007/rulingWdTally.html .  The csv file was obtained through the dashboard provided by Postgres and, alternatively, through commands from the terminal line. The python program was implemented on a ipython notebook then transferred to a .py file.

## 1) SQL QUERY:

In [ ]:
import psycopg2

conn = psycopg2.connect(**{
         'dbname':'dbname',
         'host':'dbname.awsaddress.usa-location-3.rds.amazonaws.com',
         'user':'RDSserverUser',
         'password':'password'
         })

cur = conn.cursor()

cur.execute('SELECT COUNT(DISTINCT judgeColumn) FROM "TABLE"')
print("The total number of different Court Judgments is:" , cur.fetchone()[0])

#OUTPUT:    The total number of different Court Judgments is: 938

## 2) PYTHON PROGRAM: 

In [ ]:
#for SQL Database access to Tables:
import psycopg2

#for pythonic activities:
import pandas as pd
from pandas import *
import numpy as np
import sys

# for AWS connections:
import boto
from boto.s3.connection import S3Connection
from boto.s3.key import Key
from collections import Counter

conn = psycopg2.connect(**{
         'dbname':'dbname',
         'host':'dbname.awsaddress.usa-location-3.rds.amazonaws.com',
         'user':'RDSserverUser',
         'password':'password'
         })

cur = conn.cursor()


#get rows with unique Judicial rulings from the database
# bold column has at least one judicial ruling rendered in the case
cur.execute('SELECT bold FROM "CASE2"')
#cur.execute('SELECT COUNT(DISTINCT bold) FROM "CASE2"')

#put SQL querried rows in a pandas dataframe
rows = cur.fetchall()
### Below is random exploration of the SQL to PYTHON (pandas) Experience
#rowsDF = pd.DataFrame(rows)
#print(rowsDF)
#print(rows)
#print(rows[3])
##cur.execute('ROLLBACK')
#rowsDF = pd.DataFrame(rows)
#type(rows)
#print(rowsDF[0])

#print(rows) #a list
str1 = str(rows) #a string
#type(str1)
every_word = str1.split(" ")
rule_wdcount = Counter(every_word)
rulings = rule_wdcount.most_common(150)
rule_df=pd.DataFrame(rulings, columns = ['WORD', 'FREQUENCY']) #pandas
# print(rule_df)

## Convert DF to HTML
rule_df.to_html('rulingWdTally.html')
#Counter(rowsDF[0])

# Connecting to Desired AWS S3 bucket
connS3 = S3Connection('AWSACCESSKEY', 'AWSSECRETKEY')  #need con
postingBucket = connS3.get_bucket('output6007')

# Tossing-up Output (Wd Ct for Judicial Rulings in dB) on the (AWS) web
postingBucket.get_all_keys()
file_key = postingBucket.new_key('rulingWdTally.html')

file_key.content_type = 'text/html'
file_key.set_contents_from_filename('rulingWdTally.html', policy='public-read')

Here is the link for the output:
https://s3.amazonaws.com/output6007/rulingWdTally.html

![](JUSTICEcomputer.png)

## LESSONS LEARNED & TO DO LIST:

*  All 5 Goals of: Data Acquisition, Storage, Structure, Synthesizing, and Show have been accomplished within a very short timeframe.


*  BeautifulSoup can also be a tool for parsing for XML files, the standard format for the legal support industry.


*  Data Sets from third parties are a legal form of intellectual proerty, namely trade secrets.  Obaining permission to access such datasets from others, is nearly imposssible.  Assume that the process will take quite a long time, and set your general expectations toward never recieving any such dataset to keep ones expectations "in-check".

   
*  A common saying in the data science industry is that "A DATASET PROJECT REQUIRES 80% CLEANING AND 20% APPLICATION".  As these law cases were originally entered by hand by court clerks over past one hundred years onto a legal database, there were many typos, duplicates, and as well as omissions that requires meticulous, time consuming attention.
   
   
* Continue to complete cleaning the immigration case data currently stored on AWS S3.  Torchlight Legal will then continue to develop machine learning models to assist volunteer attorneys with completing their legal research faster and thus taking on more underserved legal clients -- and winning more cases in court.


    
    
    
    

    
    
    

![](TorchlightL.png)

## OUTCOMES -- 2.0 VERSION ARCHITECTURE [Optional]:

* As more than one RDS database can be applied to a single EC2 instance (up to 1Tb), Torchlight Legal will work with OneJustice in San Francisco to build a database that stores information retrieved by legal service request applications provided by potential clients on the Torchlight Legal phone application.  Often, client intake applications are filled-out with paper forms on clipboards.


* Torchlight Legal will work with "developer-activists" who want to impactfully take action to ensure that the benefits of our democracy will be enjoyed by all.  Accordingly, a "Torchlight-Legal" Github repo has been created to build out a front-end interface for Torchlight Legal so that any individual may potentially have direct access to legal representation.


*  Presently, both the University of Texas - Austin and the Austin Tech Alliance have shown a great willingness to develop an open-source framework for creating simple legal intake forms for Torchlight Legal to implement in conjunction with those pro bono legal service entitles sanctioned by the State Bar of Texas.


*  Innovation Law Labs (Portland, Oregon) is interested in adding documents that have proven highly successful in U.S. immigration courts for purposes of generating better machine learning models.  Ultimately, Innovation Law Labs wants to broaden its assistance to legally underserved populations, and Torchlight Legal intends to assist in those efforts.



To help afflicted immigrants, the below recent photo shows's OneJustice's "pop-up" legal clinic established at San Francisco International Airport on the weekend after the 27 January 2017 travel ban was put in effect by the Trump Administration. 

![](SFairportLawFirm.png)

[OPTIONAL - Start a Company]: 

Torchlight Legal has been legally incorporated.  Importantly, we are asking volunteer developers who want to take action to ensure that positive changes to the democratic system are for the good of all that share in this bounty of our great nation.  Torchlight Legal intends to create a USER INTERFACE for client intake (electronic forms and client information database development) for the phone app. so that vulnerable populations may receive legal services to thus expand and protect the human rights of all. 

https://github.com/Torchlight-Legal/Justice

    
## RESOURCES --- CROWDSOURCING JUSTICE:


PODCAST - A panel discussion of how Tech may be impactfully applied to Justice System:
* Crowdsourcing Justice - SXSW 2017 by SXSW on SoundCloud
https://m.soundcloud.com/officialsxsw/crowdsourcing-justice-sxsw-2017 


ARTICLES - Recent Activity in Justice Tech:
* TEXAS BAR BLOG:
    http://blog.texasbar.com/2017/03/articles/sxsw/sxsw-panel-how-attorneys-are-crowdsourcing-to-increase-access-to-justice/
    
    
* AUSTIN TECH ALLIANCE:
    https://medium.com/@AustinTechAll/guest-post-recapping-sxsws-crowdsourcing-justice-panel-40799888caa4
        

* All photos above used under Fair Use Doctrine.